# Flipkart EDA assignment

## TODO: Upload csv file before moving to next

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Flipkart Data Engineering").getOrCreate()
file_path = '/FileStore/tables/Flipkart.csv'
flipkart_df = spark.read.csv(file_path, header=True, inferSchema=True)

In [0]:
flipkart_df.printSchema()
flipkart_df.count()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- maincateg: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- actprice1: integer (nullable = true)
 |-- norating1: integer (nullable = true)
 |-- noreviews1: integer (nullable = true)
 |-- star_5f: integer (nullable = true)
 |-- star_4f: integer (nullable = true)
 |-- star_3f: integer (nullable = true)
 |-- star_2f: integer (nullable = true)
 |-- star_1f: integer (nullable = true)
 |-- fulfilled1: integer (nullable = true)

Out[2]: 5244

In [0]:
flipkart_df.show(5)

+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|   id|               title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| 2242|Casuals For Men  ...|   3.8|      Men|Flipkart|      999|    27928|      3543|  14238|   4295|   3457|   1962|   3976|         1|
|20532|Women Black Flats...|   3.9|    Women|Flipkart|      499|     3015|       404|   1458|    657|    397|    182|    321|         1|
|10648|Women Gold Wedges...|   3.9|    Women|Flipkart|      999|      449|        52|    229|     70|     71|     33|     46|         1|
|20677|Men's Height Incr...|   3.9|      Men|Flipkart|     2999|      290|        40|    141|     51|     49|     17|     32|         1|
|12593|Loafers For Men  ...|   3.9|      

In [0]:
# Display all category names
flipkart_df.columns

Out[7]: ['id',
 'title',
 'Rating',
 'maincateg',
 'platform',
 'actprice1',
 'norating1',
 'noreviews1',
 'star_5f',
 'star_4f',
 'star_3f',
 'star_2f',
 'star_1f',
 'fulfilled1']

In [0]:
# Filter products with rating > 4.0 and more than 1000 reviews

filtered_df = flipkart_df.filter((flipkart_df["Rating"] > 4.0) & (flipkart_df["noreviews1"] > 1000))

# Show the results
filtered_df.show()

+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|   id|               title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| 3205|     SFG-23 Slippers|   4.1|      Men|Flipkart|      319|    12936|      1598|   7030|   2850|   1453|    587|   1016|         1|
|15762|Running Shoes For...|   4.1|      Men|Flipkart|      999|    57372|      8254|  32715|  10686|   6362|   2806|   4803|         0|
| 7561|Men Navy Sports S...|   4.2|      Men|Flipkart|      499|    17035|      2103|   9223|   4219|   1888|    614|   1091|         1|
|11078|Men Black Clogs S...|   4.2|      Men|Flipkart|     2995|    12390|      1098|   6891|   3100|   1035|    429|    935|         1|
|13206|Perfect Stylish,T...|   4.2|    Wo

In [0]:
# Display Products in 'Men' category that are fulfilled
men_fulfilled_df = flipkart_df.select("maincateg","fulfilled1").filter((flipkart_df["maincateg"] == "Men") & (flipkart_df["fulfilled1"] == 1))
men_fulfilled_df.show()

+---------+----------+
|maincateg|fulfilled1|
+---------+----------+
|      Men|         1|
|      Men|         1|
|      Men|         1|
|      Men|         1|
|      Men|         1|
|      Men|         1|
|      Men|         1|
|      Men|         1|
|      Men|         1|
|      Men|         1|
|      Men|         1|
|      Men|         1|
|      Men|         1|
|      Men|         1|
|      Men|         1|
|      Men|         1|
|      Men|         1|
|      Men|         1|
|      Men|         1|
|      Men|         1|
+---------+----------+
only showing top 20 rows



In [0]:
# Dsiplay number of products per category
flipkart_df.groupBy("maincateg").count().show()


+---------+-----+
|maincateg|count|
+---------+-----+
|     null|   67|
|      Men| 2145|
|    Women| 3032|
+---------+-----+



In [0]:
# Display Average rating per category
from pyspark.sql.functions import avg
flipkart_df.groupBy("maincateg").agg(avg("Rating").alias("average_rating")).show()

+---------+------------------+
|maincateg|    average_rating|
+---------+------------------+
|     null|             4.225|
|      Men|3.9662970106075246|
|    Women|4.0416582064297915|
+---------+------------------+



In [0]:
# Dsiplay Category with highest average number of reviews
from pyspark.sql.functions import avg
flipkart_df.groupBy("maincateg").agg(avg("noreviews1").alias("Avg_reviews")).orderBy("Avg_reviews",ascending=False).show(1)

+---------+-----------------+
|maincateg|      Avg_reviews|
+---------+-----------------+
|      Men|759.6554778554779|
+---------+-----------------+
only showing top 1 row



In [0]:
# Top 5 products with highest price. display product name and price
from pyspark.sql.functions import desc
flipkart_df.orderBy(desc("actprice1")).select("title","actprice1").show(5)

+--------------------+---------+
|               title|actprice1|
+--------------------+---------+
|GEL-QUANTUM 360 K...|    15999|
|Women Orange Spor...|    12999|
|Hybrid Astro Runn...|    10999|
|GT-2000 5 (D) Run...|    10999|
|DEERUPT RUNNER W ...|     9999|
+--------------------+---------+
only showing top 5 rows



In [0]:
# Display Min, max, and avg price per category
from pyspark.sql.functions import min,max,avg
flipkart_df.groupBy('maincateg').agg(min('actprice1').alias('min_price'),max('actprice1').alias('min_price'),avg('actprice1').alias('min_price')).show()

+---------+---------+---------+------------------+
|maincateg|min_price|min_price|         min_price|
+---------+---------+---------+------------------+
|     null|      499|     5999|1574.7313432835822|
|      Men|      139|    10999|  1455.86013986014|
|    Women|      162|    15999|1319.7081134564644|
+---------+---------+---------+------------------+



In [0]:
# Display number of nulls in each column
from pyspark.sql.functions import col, sum, when
null_counts = flipkart_df.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in flipkart_df.columns]).show()

+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| id|title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|  0|    0|   203|       67|       0|        0|        0|         0|     68|      0|      0|      0|    186|         0|
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+



In [0]:
# calculate and display the category name, number of fulfilled, and unfulfilled products
flipkart_df.groupBy("maincateg").agg(
    count(when(col("fulfilled1") == "1", True)).alias("Fulfilled"),
    count(when(col("fulfilled1") != "1", True)).alias("Unfulfilled")
).show()

+---------+---------+-----------+
|maincateg|Fulfilled|Unfulfilled|
+---------+---------+-----------+
|     null|       40|         27|
|      Men|     1352|        793|
|    Women|     1778|       1254|
+---------+---------+-----------+



In [0]:
# Display Count of products per category
display(flipkart_df.groupBy("maincateg").count())

maincateg,count
null,67
Men,2145
Women,3032


In [0]:
# Display Average rating per category
from pyspark.sql.functions import avg
display(flipkart_df.groupBy("maincateg").agg(avg("Rating").alias("average_rating")))

maincateg,average_rating
null,4.225
Men,3.9662970106075246
Women,4.0416582064297915


In [0]:
# Display Category with highest average number of reviews
from pyspark.sql.functions import avg
display(flipkart_df.groupBy("maincateg").agg(avg("noreviews1").alias("Avg_reviews")).orderBy("Avg_reviews",ascending=False).limit(1))

maincateg,Avg_reviews
Men,759.6554778554779


In [0]:
# Display Bar chart of product count per category

# Use Databricks UI to visualize this as a bar chart
display(flipkart_df.groupBy("maincateg").count())


maincateg,count
null,67
Men,2145
Women,3032


Databricks visualization. Run in Databricks to view.

In [0]:
# Bar chart of average rating per category
from pyspark.sql.functions import avg
display(flipkart_df.groupBy("maincateg").agg(avg("Rating").alias("average_rating")))

maincateg,average_rating
null,4.225
Men,3.9662970106075246
Women,4.0416582064297915


Databricks visualization. Run in Databricks to view.

In [0]:
# Display Bar chart of total number of reviews per category
display(flipkart_df.groupBy("maincateg").agg(sum("noreviews1").alias("total_reviews")))

maincateg,total_reviews
null,14084
Men,1629461
Women,535290


Databricks visualization. Run in Databricks to view.

In [0]:
# Display product name and 5 star rating for those products which have highest 5 star rating


from pyspark.sql.functions import max

# Step 1: Get the max value of 5-star ratings
# max_5_star = flipkart_df.agg(max('star_5f').alias('max_5_star')).collect()[0]['max_5_star']
max_5_star = flipkart_df.select(max('star_5f')).first()[0]
# Step 2: Filter products with that max 5-star rating and display name and rating
display(flipkart_df.filter(flipkart_df['star_5f'] == max_5_star) .select('title', 'star_5f') )

title,star_5f
"Jio-13 Running Shoes For Men (Black, Grey)",151193
"Running Shoes For Men (Silver, Black)",151193
